In [ ]:
import sys
import math
import psycopg2
import psycopg2.extras
import numpy as np
from matplotlib import pyplot
from astropy import units
from astropy.coordinates import SkyCoord
import requests

## Connect to DESI Postgres Database

In [ ]:
dbhost = "decatdb.lbl.gov"
dbname = "desi"
dbuser = "desi"
dbpasswd = "<put right password here>"

secrets_file = '/global/cfs/cdirs/desi/science/td/secrets/desi_pg.txt'
with open( secrets_file, 'r' ) as f:
    data = f.read()
    dbname, duser, dbpasswd, dbhost = data.split()

In [ ]:
dbconn = psycopg2.connect( f"dbname='{dbname}' host='{dbhost}' user='{dbuser}' password='{dbpasswd}'",
                         cursor_factory=psycopg2.extras.RealDictCursor )

## Test dates

In [ ]:
date_gt = '2021-09-17'
date_lt = '2021-09-18'

In [ ]:
radius = 1./3600.
inv_prec = 100

## MARS query

In [ ]:
mq= requests.get(f'https://mars.lco.global/?format=json&sort_value=jd&time__gt={date_gt}&time__lt={date_lt}')

In [ ]:
results = mq.json()['results']

In [ ]:
cursor = dbconn.cursor()
n=0
for result in results:
    if n>5:
        break
    q = ( "SELECT mean_fiber_ra, mean_fiber_dec, targetid, z,zerr FROM everest.zpix_redshifts "
          "WHERE zwarn=0 and q3c_radial_query( mean_fiber_ra, mean_fiber_dec, %(ra)s, %(dec)s, %(radius)s ) " )
    cursor.execute( q, { 'ra': result['candidate']['ra'],
                         'dec': result['candidate']['dec'],
                         'radius': radius } )
    matches = cursor.fetchall()
    print(result['objectId'], result['candidate']['ra'],result['candidate']['dec'],end="")
    if len(matches) == 0:
        print()
    else:
        for match in matches:
            print(match["mean_fiber_ra"], match["mean_fiber_dec"],np.round(inv_prec*match["z"])/inv_prec, np.round(inv_prec*match["zerr"])/inv_prec, match["targetid"]) 
    n=n+1